# Shoreline extraction from satellite images

This notebook shows how to download satellite images (Landsat 5,7,8 and Sentinel-2) from Google Earth Engine and apply the shoreline detection algorithm described in *Vos K., Harley M.D., Splinter K.D., Simmons J.A., Turner I.L. (in review). Capturing intra-annual to multi-decadal shoreline variability from publicly available satellite imagery, Coastal Engineering*.  

## Initial settings

The Python packages required to run this notebook can be installed by running the following anaconda command:
*"conda env create -f environment.yml"*. This will create a new enviroment with all the relevant packages installed. You will also need to sign up for Google Earth Engine (https://earthengine.google.com and go to signup) and authenticate on the computer so that python can access via your login.

In [ ]:
import os
import pickle
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
# load modules from directory
import SDS_download, SDS_preprocess, SDS_tools, SDS_shoreline

## Download images

Define the region of interest, the dates and the satellite missions from which you want to download images. The image will be cropped on the Google Earth Engine server and only the region of interest will be downloaded resulting in low memory allocation (~ 1 megabyte/image for 5 km of beach).

In [ ]:
# define the area of interest (longitude, latitude)
polygon = [[[151.301454, -33.700754],
            [151.311453, -33.702075], 
            [151.307237, -33.739761],
            [151.294220, -33.736329],
            [151.301454, -33.700754]]]
             
# define dates of interest
dates = ['2017-12-01','2018-01-01']

# define satellite missions ('L5' --> landsat 5 , 'S2' --> Sentinel-2)
sat_list = ['L5', 'L7', 'L8', 'S2']

# give a name to the site
sitename = 'NARRA'

# download satellite images. The cropped images are saved in a '/data' subfolder. The image information is stored
# into 'metadata.pkl'.
# SDS_download.get_images(sitename, polygon, dates, sat_list)

## Shoreline extraction

Performs a sub-pixel resolution shoreline detection method integrating a supervised classification component that allows to map the boundary between water and sand.

In [ ]:
# parameters and settings
%matplotlib qt
settings = {    
    'sitename': sitename,
       
    # general parameters:
    'cloud_thresh': 0.5,        # threshold on maximum cloud cover
    'output_epsg': 28356,       # epsg code of the desired output spatial reference system
       
    # shoreline detection parameters:
    'min_beach_size': 20,       # minimum number of connected pixels for a beach
    'buffer_size': 7,           # radius (in pixels) of disk for buffer around sandy pixels
    'min_length_sl': 200,       # minimum length of shoreline perimeter to be kept 
    'max_dist_ref': 100 ,       # max distance (in meters) allowed from a reference shoreline
       
    # quality control:
    'check_detection': True     # if True, shows each shoreline detection and lets the user 
                                # decide which shorleines are correct and which ones are false due to
                                # the presence of clouds and other artefacts. 
                                # If set to False, shorelines are extracted from all images.
       }

# load metadata structure (contains information on the downloaded satellite images and is created
# after all images have been successfully downloaded)
filepath = os.path.join(os.getcwd(), 'data', settings['sitename'])
with open(os.path.join(filepath, settings['sitename'] + '_metadata' + '.pkl'), 'rb') as f:
    metadata = pickle.load(f)
    
# [OPTIONAL] saves .jpg files of the preprocessed images (cloud mask and pansharpening/down-sampling) 
#SDS_preprocess.save_jpg(metadata, settings)

# [OPTIONAL] to avoid false detections and identify obvious outliers there is the option to
# create a reference shoreline position (manually clicking on a satellite image)
settings['refsl'] = SDS_preprocess.get_reference_sl(metadata, settings)

# extract shorelines from all images. Saves out.pkl which contains the shoreline coordinates for each date in the spatial
# reference system specified in settings['output_epsg']. Save the output in a file called 'out.pkl'.
out = SDS_shoreline.extract_shorelines(metadata, settings)

## Plot the shorelines

In [ ]:
plt.figure()
plt.axis('equal')
plt.xlabel('Eastings [m]')
plt.ylabel('Northings [m]')
plt.title('Shorelines')
for satname in out.keys():
    if satname == 'meta':
        continue
    for i in range(len(out[satname]['shoreline'])):
        sl = out[satname]['shoreline'][i]
        date = out[satname]['timestamp'][i]
        plt.plot(sl[:,0], sl[:,1], '-', label=date.strftime('%d-%m-%Y'))
plt.legend()